* EDA for formulation, neighbor+-3, add [MASK] 

# Import Libraries and Data

In [1]:
import numpy as np
import pandas as pd
pd.get_option("display.max_columns")
pd.set_option('display.max_columns', 300)
pd.get_option("display.max_rows")
pd.set_option('display.max_rows', 300)

import matplotlib.pyplot as plt
%matplotlib inline

import os
from os.path import join as opj
import gc

INPUT_PATH = '../../input/feedback-prize-effectiveness/'

LABEL = 'discourse_effectiveness'

In [2]:
train_df = pd.read_csv(opj(INPUT_PATH, 'train.csv'))
test_df = pd.read_csv(opj(INPUT_PATH, 'test.csv'))
sub_df = pd.read_csv(opj(INPUT_PATH, 'sample_submission.csv'))

print('train_df.shape = ', train_df.shape)
print('test_df.shape = ', test_df.shape)
print('sub_df.shape = ', sub_df.shape)

train_df.shape =  (36765, 5)
test_df.shape =  (10, 4)
sub_df.shape =  (10, 4)


# Construct part of full text which includes discourse_text

In [3]:
unique_ids = sorted(train_df['essay_id'].unique())

In [4]:
from tqdm import tqdm

NUM_NEIGHBOR = 3

essay_df = []
for essay_id in tqdm(unique_ids):
    tmp_essay_df = train_df[train_df['essay_id']==essay_id].reset_index(drop=True)
    for i in range(len(tmp_essay_df)):
        text = ''
        tmp_df = tmp_essay_df.iloc[max(0,i-NUM_NEIGHBOR):min(i+NUM_NEIGHBOR,len(tmp_essay_df))].reset_index(drop=True)
        for j in range(len(tmp_df)):
            sample = tmp_df.iloc[j]
            #text += f'[{sample["discourse_type"].upper()}]{sample["discourse_text"]}'
            text += f'[MASK]{sample["discourse_text"]}'
        tmp_essay_df.loc[i,'neighbor_text'] = text
    essay_df.append(tmp_essay_df)
    
essay_df = pd.concat(essay_df).reset_index(drop=True)
print('essay_df.shape = ', essay_df.shape)

100%|██████████| 4191/4191 [00:30<00:00, 136.80it/s]


essay_df.shape =  (36765, 6)


In [5]:
essay_df.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,neighbor_text
0,fe6dfbd53216,00066EA9880D,Driverless cars are exaclty what you would exp...,Lead,Adequate,[MASK]Driverless cars are exaclty what you wou...
1,ca9e1b60c9fb,00066EA9880D,The developement of these cars should be stopp...,Position,Effective,[MASK]Driverless cars are exaclty what you wou...
2,6cf2157f4f19,00066EA9880D,the driver will be alerted when they will need...,Claim,Effective,[MASK]Driverless cars are exaclty what you wou...
3,d920880f4341,00066EA9880D,This is such a dangerous thing because we all ...,Evidence,Effective,[MASK]Driverless cars are exaclty what you wou...
4,7d52c3216a0e,00066EA9880D,Another thing that can go wrong with these car...,Claim,Effective,[MASK]The developement of these cars should be...


In [6]:
train_df = train_df.merge(essay_df[['discourse_id','neighbor_text']], on='discourse_id', how='left')

In [7]:
import os
os.makedirs('./result', exist_ok=True)
train_df.to_csv('./result/train.csv', index=False)